Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Age and Gender

In [110]:
from gssutils import *
if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
    inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)
    tab = loadxlstabs(inputFile, sheetids='Table 1')[0]

Loading in/data-census-drug-alcohol-treatment-services.xlsx which has size 46265 bytes
Table names: ['Table 1']


In [111]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()

In [112]:
observations

{<L6 1032.0>, <I12 100.0>, <D5 294.0>, <E8 74.1>, <I9 32.1>, <K12 28.8>, <M12 100.0>, <D10 41.2>, <M10 100.0>, <B7 64.0>, <L10 22.7>, <F8 61.9>, <D12 38.9>, <K8 73.5>, <H8 76.3>, <K6 1496.0>, <B11 5.9>, <L12 17.3>, <J9 39.2>, <C9 15.1>, <H9 23.7>, <G8 71.3>, <E11 100.0>, <C7 49.0>, <M8 68.6>, <F5 2482.0>, <C5 324.0>, <J7 1010.0>, <B6 31.0>, <F7 946.0>, <K11 36.5>, <H7 252.0>, <E6 528.0>, <H5 1062.0>, <D11 42.0>, <E9 25.9>, <G11 34.2>, <L9 23.9>, <I7 1689.0>, <E5 713.0>, <C6 275.0>, <B12 34.6>, <M6 4095.0>, <J8 60.8>, <D6 222.0>, <G6 1221.0>, <G10 32.6>, <C11 52.1>, <H12 14.9>, <L5 1356.0>, <D9 24.5>, <K9 26.5>, <L8 76.1>, <I11 100.0>, <F12 56.0>, <C10 45.4>, <M11 100.0>, <I8 67.9>, <K5 2036.0>, <F11 43.1>, <G12 29.1>, <M7 1874.0>, <G5 1712.0>, <L7 324.0>, <C12 26.5>, <G9 28.7>, <L11 25.2>, <J6 1567.0>, <J11 38.3>, <B9 67.4>, <M9 31.4>, <D7 72.0>, <I6 3567.0>, <E12 100.0>, <C8 84.9>, <E10 100.0>, <B10 13.3>, <I10 100.0>, <F6 1536.0>, <J10 43.2>, <D8 75.5>, <J5 2577.0>, <H11 22.7>, <J12 

In [113]:
age = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
age

{<M3 'Overall Total'>, <F3 '18 and over'>, <B3 'Under 18 '>, <J3 'Treatment Type'>}

In [104]:
Treatment = tab.excel_ref('B4').expand(RIGHT)
Treatment

{<F4 'Alcohol Only'>, <M4 ''>, <C4 'Drugs Only'>, <B4 'Alcohol Only'>, <E4 'Total'>, <H4 'Drugs & Alcohol'>, <I4 'Total'>, <K4 'Drugs Only'>, <J4 'Alcohol Only'>, <L4 'Drugs & Alcohol'>, <G4 'Drugs Only'>, <D4 'Drugs & Alcohol'>}

In [105]:
sex = tab.excel_ref('A5').expand(DOWN) - tab.excel_ref('A13').expand(DOWN)  
sex

{<A9 'Female (%)'>, <A8 'Male (%)'>, <A7 'Female'>, <A12 '% of all Females'>, <A10 '% of Total'>, <A11 '% of all Males '>, <A5 'Total'>, <A6 'Male'>}

In [106]:
Dimensions = [
            HDim(Treatment,'Treatment Type',CLOSEST,LEFT), # Changed from DIRECTLY,ABOVE due to merged cells - Lperryman
            HDim(sex,'Sex',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Period','1 March 2017')
            ]

In [107]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [109]:
new_table = c1.topandas()
new_table.loc[new_table['Age'] == 'Treatment Type', 'Age'] = 'All Ages'
new_table.loc[new_table['Age'] == 'Overall Total', 'Age'] = 'All Ages'
new_table.loc[new_table['Treatment Type'] == '', 'Treatment Type'] = 'Total'
new_table

,OBS,Treatment Type,Sex,Age,Measure Type,Unit,Period
0,95.0,Alcohol Only,Total,Under 18,Count,People,1 March 2017
1,324.0,Drugs Only,Total,Under 18,Count,People,1 March 2017
2,294.0,Drugs & Alcohol,Total,Under 18,Count,People,1 March 2017
3,713.0,Total,Total,Under 18,Count,People,1 March 2017
4,2482.0,Alcohol Only,Total,18 and over,Count,People,1 March 2017
5,1712.0,Drugs Only,Total,18 and over,Count,People,1 March 2017
6,1062.0,Drugs & Alcohol,Total,18 and over,Count,People,1 March 2017
7,5256.0,Total,Total,18 and over,Count,People,1 March 2017
8,2577.0,Alcohol Only,Total,All Ages,Count,People,1 March 2017
9,2036.0,Drugs Only,Total,All Ages,Count,People,1 March 2017


In [79]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [80]:
new_table.dtypes

Value             float64
Treatment Type     object
Sex                object
Age                object
Measure Type       object
Unit               object
Period             object
dtype: object

In [81]:
new_table.tail(5)

,Value,Treatment Type,Sex,Age,Measure Type,Unit,Period
91,100.0,Total,% of all Females,18 and over,Count,People,1 March 2017
92,53.9,Alcohol Only,% of all Females,All Ages,Count,People,1 March 2017
93,28.8,Drugs Only,% of all Females,All Ages,Count,People,1 March 2017
94,17.3,Drugs & Alcohol,% of all Females,All Ages,Count,People,1 March 2017
95,100.0,Overall Total,% of all Females,Overall Total,Count,People,1 March 2017


In [82]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [83]:
def user_perc(x):
    
    if str(x) == 'Total':
        return 'Persons'
    else:
        return x
    
new_table['Sex'] = new_table.apply(lambda row: user_perc(row['Sex']), axis = 1)


In [84]:
new_table['Treatment Type'].fillna('All', inplace = True)
new_table['Service Type'] = 'All'
new_table['Residential Status'] = 'All'
new_table['Health and Social Care Trust']  = 'All'

In [85]:
new_table.head(1)

,Value,Treatment Type,Sex,Age,Measure Type,Unit,Period,Service Type,Residential Status,Health and Social Care Trust
0,95.0,Alcohol Only,Persons,Under 18,Count,People,1 March 2017,All,All,All


In [86]:
new_table = new_table[['Period', 'Sex', 'Age', 'Service Type', 'Residential Status', 'Treatment Type', 'Health and Social Care Trust', 'Measure Type', 'Unit', 'Value']]

In [89]:
new_table

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,All,All,Alcohol Only,All,Count,People,95.0
1,1 March 2017,Persons,Under 18,All,All,Drugs Only,All,Count,People,324.0
2,1 March 2017,Persons,Under 18,All,All,Drugs & Alcohol,All,Count,People,294.0
3,1 March 2017,Persons,Under 18,All,All,Total,All,Count,People,713.0
4,1 March 2017,Persons,18 and over,All,All,Alcohol Only,All,Count,People,2482.0
5,1 March 2017,Persons,18 and over,All,All,Drugs Only,All,Count,People,1712.0
6,1 March 2017,Persons,18 and over,All,All,Drugs & Alcohol,All,Count,People,1062.0
7,1 March 2017,Persons,18 and over,All,All,Total,All,Count,People,5256.0
8,1 March 2017,Persons,All Ages,All,All,Alcohol Only,All,Count,People,2577.0
9,1 March 2017,Persons,All Ages,All,All,Drugs Only,All,Count,People,2036.0
